<a href="https://colab.research.google.com/github/FlexTaco/cs370/blob/main/assignments/assignment-2b/Assignment2b.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install anomalib
!pip install lightning kornia torchmetrics torch FrEIA python-dotenv open-clip-torch

  Using cached lightning-2.4.0-py3-none-any.whl.metadata (38 kB)
  Using cached FrEIA-0.2-py3-none-any.whl
  Using cached open_clip_torch-2.29.0-py3-none-any.whl.metadata (31 kB)
  Using cached pytorch_lightning-2.4.0-py3-none-any.whl.metadata (21 kB)
Using cached lightning-2.4.0-py3-none-any.whl (810 kB)
Using cached open_clip_torch-2.29.0-py3-none-any.whl (1.5 MB)
Using cached pytorch_lightning-2.4.0-py3-none-any.whl (815 kB)


In [10]:
from anomalib.data import MVTec
from anomalib.engine import Engine
from anomalib.models import Patchcore, EfficientAd
from sklearn.metrics import roc_auc_score, roc_curve
import matplotlib.pyplot as plt
import shutil
import os

categories = ['tile', 'leather', 'grid']
for category in categories:
    dataset_path = os.path.join(os.path.expanduser("~"), ".anomalib", "datasets", "MVTec", category)
    if os.path.exists(dataset_path):
        shutil.rmtree(dataset_path)
        print(f"Deleted existing dataset folder for category: {category}")

#uncomment to run for the other categories, runs too slow for the google free tier
datamodule_tile = MVTec(category='tile', train_batch_size=1)
# datamodule_leather = MVTec(category='leather', train_batch_size=1)
# datamodule_grid = MVTec(category='grid', train_batch_size=1)

#making the model, patchcore will take 2-3hrs even with disabling coreset sampling
#EfficientAd takes 40minutes
# patchcore_model = Patchcore(coreset_sampling_ratio=0.0)
efficientad_model = EfficientAd()

# patchcore_engine = Engine(max_epochs=1)
efficientad_engine = Engine(max_epochs=1)

# Train PatchCore on each category
# for data in [datamodule_tile, datamodule_leather, datamodule_grid]:
#   patchcore_engine.fit(datamodule=data, model=patchcore_model)

# # Train EfficientAD on each category
# for data in [datamodule_tile, datamodule_leather, datamodule_grid]:
#   efficientad_engine.fit(datamodule=data, model=efficientad_model)

/usr/local/lib/python3.10/dist-packages/kornia/feature/lightglue.py:44: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  @torch.cuda.amp.custom_fwd(cast_inputs=torch.float32)


In [10]:
# patchcore_engine.fit(datamodule=datamodule_tile, model=patchcore_model)
efficientad_engine.fit(datamodule=datamodule_tile, model=efficientad_model)

INFO: GPU available: False, used: False
INFO:lightning.pytorch.utilities.rank_zero:GPU available: False, used: False
INFO: TPU available: False, using: 0 TPU cores
INFO:lightning.pytorch.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO: HPU available: False, using: 0 HPUs
INFO:lightning.pytorch.utilities.rank_zero:HPU available: False, using: 0 HPUs
/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:617: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
INFO: 
  | Name                  | Type                     | Params | Mode 
---------------------------------------------------------------------------
0 | model              

Training: |          | 0/? [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/anomalib/models/image/efficient_ad/lightning_model.py:98: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
/usr/local/lib/python3.10/dist-packages/torchv

Validation: |          | 0/? [00:00<?, ?it/s]

Calculate Validation Dataset Quantiles: 100%|██████████| 4/4 [02:15<00:00, 33.98s/it]
INFO: `Trainer.fit` stopped: `max_epochs=1` reached.
INFO:lightning.pytorch.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=1` reached.


In [14]:
from sklearn.metrics import roc_auc_score, roc_curve
import matplotlib.pyplot as plt
def evaluate_model(model, datamodule):
    predictions, labels = [], []

    for batch in datamodule.test_dataloader():
        output = model(batch["image"])
        predictions.extend(output["predictions"].cpu().numpy())
        labels.extend(batch["label"].cpu().numpy())

    auroc_score = roc_auc_score(labels, predictions)
    print(f"AUROC Score for EfficientAD on 'tile' category: {auroc_score:.2f}")


evaluate_model(efficientad_model, datamodule_tile)

AttributeError: 'MVTec' object has no attribute 'test_data'